In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<img src='../../../common/logo_DH.png' align='left' width=35%/>

# BigQuery ML

## Intro

Clustering K-means

https://cloud.google.com/bigquery-ml/docs/kmeans-tutorial

BigQuery ML admite aprendizaje no supervisado. 

Podemos aplicar el algoritmo k-means para agrupar datos en clusters. 

Usaremos un modelo de k-means en BigQuery ML para crear clústeres de datos en el conjunto de datos públicos de Alquileres de bicicletas de Londres (https://console.cloud.google.com/marketplace/details/greater-london-authority/london-bicycles). 

Los datos de Alquileres de bicicletas de Londres contienen la cantidad de alquileres del Esquema de alquileres de bicicletas Santander de Londres desde 2011 hasta el presente. 

Los datos incluyen marcas de tiempo de inicio y parada, nombres de estaciones y duración del viaje.

Las consultas usan funciones geográficas (https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions) disponibles en BigQuery GIS. Si desean obtener más información sobre BigQuery GIS, consulten Introducción a BigQuery GIS (https://cloud.google.com/bigquery/docs/gis-intro).

## Ejercicio 1

En la página de selección de proyectos de Cloud Console, selecciona o crea un proyecto de Cloud.

https://console.cloud.google.com/projectselector2/home/dashboard

(Puede ser clase-37)

Crear un dataset `bqml_tutorial` para almacenar el modelo. En Ubicación de datos (Data location), seleccionemos European Union (EU).
El conjunto de datos públicos de Alquileres de bicicletas de Londres se almacena en la ubicación multiregión EU.


Solución:

https://cloud.google.com/bigquery-ml/docs/kmeans-tutorial#step_one_create_your_dataset

## Ejercicio 2

Agruparemos en clusters las estaciones de bicicletas en función de los siguientes atributos:

* Duración de los alquileres
* Cantidad de viajes por día
* Distancia desde el centro de la ciudad


Examinemos los datos de entrenamiento ejecutando esta consulta en https://console.cloud.google.com/bigquery

<code>
WITH
  hs AS (
  SELECT
    h.start_station_name AS station_name,
    IF
    (EXTRACT(DAYOFWEEK
      FROM
        h.start_date) = 1
      OR EXTRACT(DAYOFWEEK
      FROM
        h.start_date) = 7,
      "weekend",
      "weekday") AS isweekday,
    h.duration,
    ST_DISTANCE(ST_GEOGPOINT(s.longitude,
        s.latitude),
      ST_GEOGPOINT(-0.1,
        51.5))/1000 AS distance_from_city_center
  FROM
    `bigquery-public-data.london_bicycles.cycle_hire` AS h
  JOIN
    `bigquery-public-data.london_bicycles.cycle_stations` AS s
  ON
    h.start_station_id = s.id
  WHERE
    h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
  stationstats AS (
  SELECT
    station_name,
    AVG(duration) AS duration,
    COUNT(duration) AS num_trips,
    MAX(distance_from_city_center) AS distance_from_city_center
  FROM
    hs
  GROUP BY
    station_name )
SELECT
  *
FROM
  stationstats
ORDER BY
  distance_from_city_center ASC
</code>


Esta consulta extrae datos sobre el alquiler de bicicletas, incluidos start_station_name y duration, y los une con la información de la estación, incluida distance-from-city-center. 

Luego, calcula los atributos de la estación en stationstats, incluida la duración promedio de los viajes y la cantidad de viajes, y pasa por el atributo de estación distance_from_city_center.

En esta consulta, se usa `WITH` para definir subconsultas. 

También se usan las funciones de BigQuery GIS ST_DISTANCE y ST_GEOGPOINT.

Solución:

https://cloud.google.com/bigquery-ml/docs/kmeans-tutorial#step_two_examine_your_training_data

## Ejercicio 3

Crear un modelo de k-means

Podemos crear y entrenar un modelo k-means mediante la declaración `CREATE MODEL` con la opción `model_type=kmeans`. 

En la siguiente consulta, se agrega una declaración CREATE MODEL a la consulta anterior y se quitan los campos id en los datos.

Ejecutemosla en https://console.cloud.google.com/bigquery

<code>
CREATE OR REPLACE MODEL
  bqml_tutorial.london_station_clusters OPTIONS(model_type='kmeans',
    num_clusters=4) AS
WITH
  hs AS (
  SELECT
    h.start_station_name AS station_name,
  IF
    (EXTRACT(DAYOFWEEK
      FROM
        h.start_date) = 1
      OR EXTRACT(DAYOFWEEK
      FROM
        h.start_date) = 7,
      "weekend",
      "weekday") AS isweekday,
    h.duration,
    ST_DISTANCE(ST_GEOGPOINT(s.longitude,
        s.latitude),
      ST_GEOGPOINT(-0.1,
        51.5))/1000 AS distance_from_city_center
  FROM
    `bigquery-public-data.london_bicycles.cycle_hire` AS h
  JOIN
    `bigquery-public-data.london_bicycles.cycle_stations` AS s
  ON
    h.start_station_id = s.id
  WHERE
    h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
  stationstats AS (
  SELECT
    station_name,
    isweekday,
    AVG(duration) AS duration,
    COUNT(duration) AS num_trips,
    MAX(distance_from_city_center) AS distance_from_city_center
  FROM
    hs
  GROUP BY
    station_name, isweekday)
SELECT
  * EXCEPT(station_name, isweekday)
FROM
  stationstats
</code>

La declaración `CREATE MODEL` especifica la cantidad deseada de clusters: cuatro. 

En la declaración `SELECT`, `EXCEPT` excluye la columna station_name porque station_name no es un atributo. La consulta crea una fila única por station_name y en la declaración SELECT solo se mencionan los atributos.

Si omitimos la opción num_clusters, BigQuery ML elegirá un valor predeterminado razonable según la cantidad total de filas en los datos de entrenamiento. 

También podemos realizar el ajuste de hiperparámetros para encontrar una cantidad apropiada. Con el fin de determinar una cantidad óptima de clústeres, deberíamos ejecutar la consulta CREATE MODEL para diferentes valores de num_clusters, buscar la medida de error y elegir el punto en el que la medida de error esté en su valor mínimo. Podemos obtener la medida de error seleccionando un modelo y haciendo click en la pestaña Entrenamiento.


En la IU web de BigQuery, en la sección Recursos, expandiendo el nombre del proyecto, y haciendo click en bqml_tutorial y, luego en london_station_clusters vemos el modelo entrenado.

Haciendo click en la pestaña Esquema (Schema) vemos que el modelo enumera los tres atributos de estación que BigQuery ML usó para realizar el agrupamiento. 

Haciendo click en la pestaña Evaluación (Evaluation) vemos visualizaciones de los clusters identificados por el modelo k-means. En Numerical features, los gráficos de barras muestran hasta 10 de los valores de atributos numéricos más importantes para cada centroide. Podemos seleccionar qué atributos visualizar en el menú desplegable.

Solución:
    
https://cloud.google.com/bigquery-ml/docs/kmeans-tutorial#step_three_create_a_k-means_model

## Ejercicio 4

Predecir a qué cluster pertenece una estación.

Para identificar el cluster al que pertenece una estación en particular, usamos la función `ML.PREDICT`. La siguiente consulta predice el clúster de cada estación que tiene la string "Kennington" en su nombre.

<code>
WITH
  hs AS (
  SELECT
    h.start_station_name AS station_name,
    IF
    (EXTRACT(DAYOFWEEK
      FROM
        h.start_date) = 1
      OR EXTRACT(DAYOFWEEK
      FROM
        h.start_date) = 7,
      "weekend",
      "weekday") AS isweekday,
    h.duration,
    ST_DISTANCE(ST_GEOGPOINT(s.longitude,
        s.latitude),
      ST_GEOGPOINT(-0.1,
        51.5))/1000 AS distance_from_city_center
  FROM
    `bigquery-public-data.london_bicycles.cycle_hire` AS h
  JOIN
    `bigquery-public-data.london_bicycles.cycle_stations` AS s
  ON
    h.start_station_id = s.id
  WHERE
    h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
  stationstats AS (
  SELECT
    station_name,
    AVG(duration) AS duration,
    COUNT(duration) AS num_trips,
    MAX(distance_from_city_center) AS distance_from_city_center
  FROM
    hs
  GROUP BY
    station_name )
SELECT
  * EXCEPT(nearest_centroids_distance)
FROM
  ML.PREDICT( MODEL bqml_tutorial.london_station_clusters,
    (
    SELECT
      *
    FROM
      stationstats
    WHERE
      REGEXP_CONTAINS(station_name, 'Kennington')))
    
</code>

En esta consulta, se usa la función `REGEXP_CONTAINS` para buscar todas las entradas en la columna station_name que contienen la string "Kennington". La función `ML.PREDICT` usa esos valores para predecir qué clusters contendrían esas estaciones.

Solución:

https://cloud.google.com/bigquery-ml/docs/kmeans-tutorial#use_ml.predict

## Ejercicio 5

Tratar de interpretar los diferentes clústeres usando los gráficos de barras de los resultados de la evaluación del modelo y responder:

* Supongamos que necesitas experimentar con un tipo nuevo de anclaje. ¿Qué clúster de estaciones deberías elegir para realizar este experimento? 

* Supongamos que deseas abastecer algunas estaciones con bicicletas de carrera. ¿Qué estaciones debes elegir? 

Solución:

https://cloud.google.com/bigquery-ml/docs/kmeans-tutorial#step_five_use_your_model_to_make_data-driven_decisions

Cluster#3 muestra una estación de la ciudad muy demandada que se encuentra cerca del centro de la ciudad. 

Cluster#2 muestra la segunda estación de la ciudad que es menos demandada. 

Cluster#1 muestra una estación lejos del centro de la ciudad, menos demandada, con alquileres de mayor duración. 

Cluster#4 muestra otra estación lejos del centro de la ciudad, con pocos viajes que son más cortos. 

En función de estos resultados, podemos usar los datos para tomar decisiones fundamentadas. Por ejemplo:

Supongamos que necesitas experimentar con un tipo nuevo de anclaje. 
¿Qué clúster de estaciones deberías elegir para realizar este experimento? 
Las estaciones en Cluster#1, Cluster#2 o Cluster#4 parecen elecciones lógicas porque no son las estaciones más concurridas.

Supongamos que deseas abastecer algunas estaciones con bicicletas de carrera. ¿Qué estaciones debes elegir? Cluster#1 es el grupo de estaciones que están lejos del centro de la ciudad y tienen los viajes más largos. Estas son las posibles candidatas para las bicicletas de carrera.

## Referencia

https://cloud.google.com/bigquery-ml/docs/kmeans-tutorial    